In [14]:
import config
import requests
import json
import math
from prettytable import ALL as pt
import pyinputplus as pyin
import pyspark as ps
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark.sql.functions import desc, max, min
import pymysql

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
#For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.``
spark = SparkSession.builder.appName('BankTransactions').getOrCreate()
branchdf = spark.read.json(r"/Users/roy/Documents/Data-Engineer/env/capstone_DE/DE_capstone_evn/Credit_Card_Dataset/Data-Engineer-Capstone/cdw_sapp_branch.json")
creditdf = spark.read.json(r"/Users/roy/Documents/Data-Engineer/env/capstone_DE/DE_capstone_evn/Credit_Card_Dataset/Data-Engineer-Capstone/cdw_sapp_credit.json")
customerdf = spark.read.json(r"/Users/roy/Documents/Data-Engineer/env/capstone_DE/DE_capstone_evn/Credit_Card_Dataset/Data-Engineer-Capstone/cdw_sapp_custmer.json")



In [15]:
import pymysql
import configparser

# Read the configuration values from the config file
config = configparser.ConfigParser()
config.read('config.ini')

# Retrieve the database configuration values
host = config.get('database', 'host')
username = config.get('database', 'username')
password = config.get('database', 'password')

# Create a connection to the database
conn = pymysql.connect(host=host, user=username, password=password)

# Create a cursor object
cur = conn.cursor()

# Create a new database
#cur.execute("CREATE DATABASE creditcard_capstone")

# Show all databases
cur.execute("SHOW DATABASES")
databaseList = cur.fetchall()
for database in databaseList:
    print(database)

cur.execute("USE creditcard_capstone")

#show all tables
cur.execute("SHOW TABLES")
tableList = cur.fetchall()
for table in tableList:
    print(table)

# Close the connection conn.close()


('creditcard_capstone',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('CDW_SAPP_BRANCH',)
('CDW_SAPP_CREDIT_CARD',)
('CDW_SAPP_CUSTOMER',)
('CDW_SAPP_loan_application',)


In [16]:
# Connect to the database
conn = pymysql.connect(host=host, user=username, password=password, database='creditcard_capstone')
cur = conn.cursor()
print(cur)

In [17]:
spark = SparkSession.builder.appName('BankTransactions').getOrCreate()
branchdf.createOrReplaceGlobalTempView('creditcard_capstone.CDW_SAPP_BRANCH')
branchdf.show(1)
creditdf.createOrReplaceGlobalTempView('creditcard_capstone.cdw_sapp_credit')
creditdf.show(1)
customerdf.createOrReplaceGlobalTempView('creditcard_capstone.cdw_sapp_customer')
customerdf.show(1)


+-----------+-----------+------------+------------+------------+-------------+----------+--------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------+-----------+------------+------------+------------+-------------+----------+--------------------+
|  Lakeville|          1|Example Bank|  1234565276|          MN| Bridle Court|     55044|2018-04-18T16:51:...|
+-----------+-----------+------------+------------+------------+-------------+----------+--------------------+
only showing top 1 row

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|201

In [18]:
#22.1) Used to check the existing account details of a customer.
def checkCusAccDetails(first_name, last_4_digits_SSN):
    sqlSelect = "SELECT * FROM cdw_sapp_customer \
        WHERE FIRST_NAME = '{}' AND RIGHT(SSN, 4) = '{}'"
    cur.execute(sqlSelect.format(first_name, last_4_digits_SSN))
    result = cur.fetchall()
    print(result)

first_name = input("Enter First Name: ")
last_4_digits_SSN = input("Enter last 4 digits of SSN: ")
checkCusAccDetails(first_name, last_4_digits_SSN)

(('4210653310395982', 'NorthOlmsted', 'United States', 'BWoodard@example.com', '(880)124-2570', 'OH', '44070', 'Beau', 'Woodard', '2018-04-21T12:49:02.000-04:00', 'ambrose', 123454202, 'Colonial Drive, 3'),)


In [19]:

# 2.2 (2) Modify the existing account details of a customer.
def modifyCusAccDetails(first_name, last_4_digits_SSN, zipcode):
    sqlUpdate = "UPDATE cdw_sapp_customer \
        SET CUST_ZIP = '{}' \
        WHERE FIRST_NAME = '{}' AND RIGHT(SSN, 4) = '{}'"
    sqlSelect = "SELECT * FROM cdw_sapp_customer \
        WHERE FIRST_NAME = '{}' AND RIGHT(SSN, 4) = '{}'"
    cur.execute(sqlUpdate.format(zipcode, first_name, last_4_digits_SSN))
    cur.execute(sqlSelect.format(first_name, last_4_digits_SSN))
    result = cur.fetchall()
    print(result)

first_name = input("Enter First Name: ")
last_4_digits_SSN = input("Enter last 4 digits of SSN: ")
zipcode = input("Enter New Zip Code: ")
modifyCusAccDetails(first_name, last_4_digits_SSN, zipcode)

(('4210653310395982', 'NorthOlmsted', 'United States', 'BWoodard@example.com', '(880)124-2570', 'OH', '11377', 'Beau', 'Woodard', '2018-04-21T12:49:02.000-04:00', 'ambrose', 123454202, 'Colonial Drive, 3'),)


In [20]:
# 2.2 (3) generate a monthly bill for a credit card number for a given month and year.
def monthlyBill(credit_card_no, month, year):
    sqlSelect = "SELECT cr.credit_card_no, cr.TRANSACTION_TYPE, cr.TRANSACTION_VALUE, cr.TIMEID \
                FROM cdw_sapp_credit_CARD cr WHERE cr.credit_card_no = {} AND cr.MONTH = {} AND cr.YEAR = {}"
    
    cur.execute(sqlSelect.format(credit_card_no, month, year))
    result = cur.fetchall()
    print(result)

credit_card_no = input("Enter Customer Credit Card #: ")
month = input("Enter Month 2 digit format: ")
year = input("Enter 4 digit year: ")
monthlyBill(credit_card_no, month, year)

(('4210653376655865', 'Bills', 7.78, '20180317'), ('4210653376655865', 'Healthcare', 12.61, '20180321'), ('4210653376655865', 'Healthcare', 25.09, '20180324'), ('4210653376655865', 'Test', 55.65, '20180312'), ('4210653376655865', 'Bills', 77.92, '20180319'))


In [21]:
#2.2 (4) display the transactions made by a customer between two dates.
def display_transactions_by_date(credit_card_no, startDate, endDate):
    sqlSelect = "SELECT c.CREDIT_CARD_NO, c.TRANSACTION_TYPE, c.TRANSACTION_VALUE, b.BRANCH_NAME FROM cdw_sapp_credit_card c \
                JOIN cdw_sapp_customer cust \
                      ON c.CREDIT_CARD_NO = cust.CREDIT_CARD_NO \
                JOIN cdw_sapp_branch b \
                    ON c.BRANCH_CODE = b.BRANCH_CODE \
                WHERE c.CREDIT_CARD_NO = {} \
                AND c.TIMEID BETWEEN '{}' AND '{}'\
                ORDER BY c.year, c.month, c.day DESC "
    cur.execute(sqlSelect.format(credit_card_no, startDate, endDate))
    result = cur.fetchall()
    print(result)
credit_card_no = input("Enter Customer Credit Card #: ")
startDate = input("Enter STARTDATE: ")
endDate = input("Enter ENDDATE: ")
display_transactions_by_date(credit_card_no, startDate, endDate)

(('4210653376655865', 'Education', 5.18, 'Example Bank'), ('4210653376655865', 'Gas', 83.58, 'Example Bank'), ('4210653376655865', 'Entertainment', 20.87, 'Example Bank'), ('4210653376655865', 'Test', 78.25, 'Example Bank'), ('4210653376655865', 'Test', 52.41, 'Example Bank'))


In [22]:
# 2,1 -1)Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
def displaytransactions(cust_zip, month, year):
    sqlSelect = "SELECT CR.TRANSACTION_VALUE, C.CUST_ZIP, CR.MONTH, CR.YEAR, CR.DAY \
        FROM cdw_sapp_customer C\
        JOIN cdw_sapp_credit_CARD CR\
        ON c.CREDIT_CARD_NO = cr.CREDIT_CARD_NO\
        WHERE c.CUST_ZIP = {} AND MONTH = {} AND YEAR = {}\
        ORDER BY DAY DESC"
   
    #cur.execute(sqlUpdate.format(middle_name, first_name, last_4_digits_SSN))
    cur.execute(sqlSelect.format(cust_zip, month, year))
    result = cur.fetchall()
    print(result)

cust_zip = input("Enter Customer Zipcode: ")
month = input("Enter Month 2 digit format: ")
year = input("Enter 4 digit year: ")
displaytransactions(cust_zip, month, year)

((83.05, '11377', 3, 2018, 28), (31.75, '11377', 3, 2018, 28), (38.58, '11377', 3, 2018, 27), (7.58, '11377', 3, 2018, 25), (89.75, '11377', 3, 2018, 22), (95.31, '11377', 3, 2018, 22), (87.41, '11377', 3, 2018, 18), (68.55, '11377', 3, 2018, 16), (4.57, '11377', 3, 2018, 13), (92.16, '11377', 3, 2018, 10), (70.61, '11377', 3, 2018, 9), (55.98, '11377', 3, 2018, 6), (84.17, '11377', 3, 2018, 5), (25.51, '11377', 3, 2018, 5), (9.58, '11377', 3, 2018, 3), (5.38, '11377', 3, 2018, 3))


In [23]:
#Functional Requirement 2.1 2) Used to display the number and total values of transactions for a given type.

def displayvalues(transaction_type):
    sqlSelect = " SELECT cr.TRANSACTION_TYPE, COUNT(TRANSACTION_VALUE) AS COUNT, SUM(TRANSACTION_VALUE) AS TOTAL_VALUE \
        FROM cdw_sapp_credit_card cr where cr.TRANSACTION_TYPE = '{}' GROUP BY 1"
    cur.execute(sqlSelect.format(transaction_type))
    result = cur.fetchall()
    print(result)

transaction_type = input("Enter Transaction Type: ")
displayvalues(transaction_type)

(('Gas', 6605, 336059.2600000003),)


In [24]:
# 2.1 3) Used to display the total values of transactions for branches in a given state.
def displaybystate(state):
    sqlSelect = "SELECT b.BRANCH_STATE, count(c.TRANSACTION_ID), SUM(TRANSACTION_VALUE) AS TOTAL_VALUE \
        FROM CDW_SAPP_BRANCH b \
        JOIN creditcard_capstone.cdw_sapp_credit_card c \
        ON b.BRANCH_CODE = c.BRANCH_CODE \
        WHERE b.BRANCH_STATE = '{}' GROUP BY 1"
    
    cur.execute(sqlSelect.format(state))
    result = cur.fetchall()
    print(result)
    #print(type(result))

state = input("Enter State 2 Letter format: ")
displaybystate(state)



(('NY', 4143, 213717.3800000001),)


In [35]:
def custDetails(last_4_digits_SSN):
    sqlSelect = "SELECT * FROM cdw_sapp_customer \
        WHERE RIGHT(SSN, 4) = '{}'"
    cur.execute(sqlSelect.format(last_4_digits_SSN))
    result = cur.fetchall()
    print(result)

last_4_digits_SSN = input("Enter last 4 digits of SSN: ")
custDetails(last_4_digits_SSN)

(('4210653310395982', 'NorthOlmsted', 'United States', 'BWoodard@example.com', '(880)124-2570', 'OH', '11377', 'Beau', 'Woodard', '2018-04-21T12:49:02.000-04:00', 'ambrose', 123454202, 'Colonial Drive, 3'),)


In [37]:
# define a function to validate user input
def validate_input(prompt, expected_type):
    while True:
        user_input = pyin.inputStr(prompt)
        try:
            validated_input = expected_type(user_input)
            return validated_input
        except ValueError:
            print(f"Invalid input. Please enter a {expected_type.__name__}.")
# define a loop for user interaction
while True:
    print("Welcome to Banking Application select an option to begin!")
    print("1. Display Customer Account Detail")
    print("2. Display transaction count and total value by type")
    print("3. Display transaction count and total value by branch state")
    print("4. Exit ------>")

    # validate user input displaytransactions(cust_zip, month, year)
    choice = validate_input("Enter your choice: ", int)
    print('You have Choice ', choice)
    if choice == 1:
        # display customer detail
        last_4_digits_SSN = validate_input("Enter the Customer Last of SSN#: ", str)
        print("You have enter ",last_4_digits_SSN)
        result = custDetails(last_4_digits_SSN)
        print(result)

    elif choice == 2:
        print('You have selected', choice)
        # display transaction count and total value by type displayvalues(transaction_type)
        TRANSACTION_TYPE = validate_input("Enter the transaction type: ", str)
        result = displayvalues(transaction_type)
        print(result)
        
    elif choice == 3:
        print('You have selected', choice)
        # display transaction count and total value by branch state displaybystate(state)
        if state == validate_input("Enter the state: ", str):
            result = displaybystate(state)
            print(result)
        else:
            print("No transactions found.")
    elif choice == 4:
        print("GoodBye!")
        # exit
        break
    else:
        print("Invalid choice.")
        

Welcome to Banking Application select an option to begin!
1. Display Customer Account Detail
2. Display transaction count and total value by type
3. Display transaction count and total value by branch state
4. Exit ------>
Enter your choice: You have Choice  3
You have selected 3
Enter the state: (('NY', 4143, 213717.3800000001),)
None
Welcome to Banking Application select an option to begin!
1. Display Customer Account Detail
2. Display transaction count and total value by type
3. Display transaction count and total value by branch state
4. Exit ------>
Enter your choice: Invalid input. Please enter a int.
Enter your choice: You have Choice  3
You have selected 3
Enter the state: No transactions found.
Welcome to Banking Application select an option to begin!
1. Display Customer Account Detail
2. Display transaction count and total value by type
3. Display transaction count and total value by branch state
4. Exit ------>
Enter your choice: You have Choice  4
GoodBye!


In [ ]:
# Close the connection 
conn.close()